In [1]:
import pandas as pd
import os 
import requests
from bs4 import BeautifulSoup
import re 
import pandas as pd
import numpy as np

In [47]:
HLTV = 'data/downloads/furia_games_hltv.csv'
pd.read_csv(HLTV)

,Unnamed: 0,Link,Download,Event,Date,Match,Maps
0,0,https://www.hltv.org/matches/2346563/furia-vs-...,https://www.hltv.orgdemo/62423/,IEM Katowice 2021,2021-02-19,FURIA vs. NIP,"Nuke, Overpass"
1,2,https://www.hltv.org/matches/2346225/mousespor...,https://www.hltv.orgdemo/62149/,cs_summit 7,2021-01-29,mousesports vs. FURIA,"Nuke, Inferno"
2,3,https://www.hltv.org/matches/2346222/furia-vs-...,https://www.hltv.orgdemo/62137/,cs_summit 7,2021-01-28,FURIA vs. Cloud9,"Nuke, Train, Inferno"
3,4,https://www.hltv.org/matches/2346216/virtuspro...,https://www.hltv.orgdemo/62091/,cs_summit 7,2021-01-26,Virtus.pro vs. FURIA,"Inferno, Overpass"
4,5,https://www.hltv.org/matches/2346212/furia-vs-...,https://www.hltv.orgdemo/62079/,cs_summit 7,2021-01-25,FURIA vs. Cloud9,"Nuke, Train, Inferno"
...,...,...,...,...,...,...,...
353,354,https://www.hltv.org/matches/2315374/isurus-vs...,https://www.hltv.orgdemo/32643/,Alienware Liga Pro Gamers Club - SEP/17,2017-09-29,Isurus vs. FURIA,"Cache, Overpass, Mirage"
354,355,https://www.hltv.org/matches/2315075/intz-vs-f...,https://www.hltv.orgdemo/32566/,Alienware Liga Pro Gamers Club - SEP/17,2017-09-28,INTZ vs. FURIA,"Cobblestone, Inferno"
355,356,https://www.hltv.org/matches/2314324/teamone-v...,https://www.hltv.orgdemo/31626/,Alienware Liga Pro Gamers Club - AUG/17,2017-09-01,TeamOne vs. FURIA,"Cobblestone, Mirage"
356,357,https://www.hltv.org/matches/2314065/intz-vs-f...,https://www.hltv.orgdemo/31610/,Alienware Liga Pro Gamers Club - AUG/17,2017-08-31,INTZ vs. FURIA,"Cobblestone, Overpass"


In [11]:
if os.path.exists(HLTV):
    add_new_matches()
    
else:
    table = create_hltv(get_team_matches())
    table.to_csv('data/downloads/furia_games_hltv.csv')

1 matches found, starting to build HLTV

Table 100.0% completed...


ValueError: Length of values (2) does not match length of index (1)

In [2]:
def get_team_matches():
    
    # Defining headers for BeautiffulSoup

    headers = {"User-Agent":
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36"}  

    # Team ID to refer

    TEAM_ID = "8297"

    # The team's match page is splitted through "offsets", the current threshold is 300 for average and to avoid possible old demos issues

    OFFSETS = [0, 100, 200, 300]

    URLs = []

    matches = []

    for offset in OFFSETS:

        # For each of set, it creates an url where we can find the matches links
        url = "https://www.hltv.org/results?offset="+str(offset)+"&content=demo&content=vod&team="+TEAM_ID
        URLs.append(url)

    for URL in URLs:

        page = requests.get(URL,headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')

        html = soup.find_all(class_='result-con')

        offset_matches = re.findall(r'href=\"/matches/(.*?)\"', str(html))
        matches.extend(offset_matches)

    links = ["https://www.hltv.org/matches/"+sublink for sublink in matches]

    df = pd.DataFrame(links, columns=['Link'])
    df.drop_duplicates(inplace=True) # for some reason there's a value being duplicated during the offset parsin
    
    return df

In [32]:
def create_hltv(matches):
    
    headers = {"User-Agent":
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36"}  
    
    df = matches

    download_links = []
    dates = []
    events = []
    versus = []
    picks = []

    step = 0
    stages = len(df['Link'].values)

    print("{} matches found, starting to build HLTV\n".format(stages))

    for link in df['Link'].values:  

        page = requests.get(link, headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')

        try:

            # Scraping download links
            download_link = re.findall(r'href=\"/download/(.*?)\"', str(soup.find_all(class_='stream-box')))[0]
            download_links.append("https://www.hltv.org"+download_link+"/")

            # Scraping match dates
            date = re.findall(r'>(.*?)</div>', str(soup.find_all(class_='date')))[0]
            dates.append(date)

            # Scraping event name
            event = re.findall(r'title="(.*?)">', str(soup.find_all(class_='event text-ellipsis')))[0]
            events.append(event)

            # Scraping teams and setting up a "versus" format 
            team_1 = re.findall(r'title="(.*?)"/>', str(soup.find_all(class_='team1-gradient')))[0]
            team_2 = re.findall(r'title="(.*?)"/>', str(soup.find_all(class_='team2-gradient')))[0]

            versus.append(team_1 +" vs. "+team_2)

            # Scraping maps played
            maps = list(set([x.capitalize() for x in re.findall(r'>(.*?)<', 
                                                                str(soup.find_all(class_='stats-menu-link')))[2:] if len(x) > 3]))
            picks.append(', '.join(map(str, maps)))

        except Exception as e:
            print(e)
            
            download_links.append(np.nan)
            dates.append(np.nan)
            events.append(np.nan)
            versus.append(np.nan)
            picks.append(np.nan)

        step += 1
        print("Table {:.1f}% completed...".format(step/stages * 100))
    
    df['Download'] = download_links
    df['Event'] = events
    df['Date'] = pd.to_datetime(dates) 
    df['Match'] = versus
    df['Maps'] = picks

    return df

In [33]:
create_hltv(new_matches)

1 matches found, starting to build HLTV

Table 100.0% completed...
['https://www.hltv.orgdemo/62423/']


,Link,Download,Event,Date,Match,Maps
0,https://www.hltv.org/matches/2346563/furia-vs-...,https://www.hltv.orgdemo/62423/,IEM Katowice 2021,2021-02-19,FURIA vs. NIP,"Overpass, nuke, Nuke"


In [4]:
def add_new_matches():
    
    hltv = pd.read_csv(HLTV)

    CURRENT_MATCHES = hltv['Link'].to_list()

    matches = get_team_matches()['Link'].to_list()
    outer_matches = [x for x in matches if x not in CURRENT_MATCHES]

    if(outer_matches == []):
        print("There's no new matches to be added!")

    else:
        new_matches = pd.DataFrame(outer_matches, columns=['Link'])
        build_new_matches = create_hltv(new_matches)

        df = pd.concat([build_new_matches, hltv])
        df.to_csv('data/downloads/furia_games_hltv.csv', index=False)

    for match in outer_matches:
        print("{} was added to HLTV".format(match.split('/')[-1]))
        
    return "Finished!"

In [34]:
pd.read_csv(HLTV)

,Link,Download,Event,Date,Match,Maps
0,https://www.hltv.org/matches/2346563/furia-vs-...,https://www.hltv.orgdemo/62423/,IEM Katowice 2021,2021-02-19 00:00:00,FURIA vs. NIP,"Overpass, nuke, Nuke"
1,https://www.hltv.org/matches/2346225/mousespor...,https://www.hltv.orgdemo/62149/,cs_summit 7,2021-01-29,mousesports vs. FURIA,"Inferno, Cobblestone"
2,https://www.hltv.org/matches/2346222/furia-vs-...,https://www.hltv.orgdemo/62137/,cs_summit 7,2021-01-28,FURIA vs. Cloud9,"Inferno, Cobblestone"
3,https://www.hltv.org/matches/2346216/virtuspro...,https://www.hltv.orgdemo/62091/,cs_summit 7,2021-01-26,Virtus.pro vs. FURIA,"Inferno, Cobblestone"
4,https://www.hltv.org/matches/2346212/furia-vs-...,https://www.hltv.orgdemo/62079/,cs_summit 7,2021-01-25,FURIA vs. Cloud9,"Inferno, Cobblestone"
...,...,...,...,...,...,...
353,https://www.hltv.org/matches/2315374/isurus-vs...,https://www.hltv.orgdemo/32643/,Alienware Liga Pro Gamers Club - SEP/17,2017-09-29,Isurus vs. FURIA,"Inferno, Cobblestone"
354,https://www.hltv.org/matches/2315075/intz-vs-f...,https://www.hltv.orgdemo/32566/,Alienware Liga Pro Gamers Club - SEP/17,2017-09-28,INTZ vs. FURIA,"Inferno, Cobblestone"
355,https://www.hltv.org/matches/2314324/teamone-v...,https://www.hltv.orgdemo/31626/,Alienware Liga Pro Gamers Club - AUG/17,2017-09-01,TeamOne vs. FURIA,"Inferno, Cobblestone"
356,https://www.hltv.org/matches/2314065/intz-vs-f...,https://www.hltv.orgdemo/31610/,Alienware Liga Pro Gamers Club - AUG/17,2017-08-31,INTZ vs. FURIA,"Inferno, Cobblestone"


In [36]:
x = "a"

x.islower()

True

In [490]:
df.to_csv('data/downloads/furia_games_hltv.csv', index=False)

In [ ]:
# Defining headers for BeautiffulSoup

headers = {"User-Agent":
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36"}  

# Team ID to refer

TEAM_ID = "8297"

# The team's match page is splitted through "offsets", the current threshold is 300 for average and to avoid possible old demos issues

OFFSETS = [0, 100, 200, 300]

URLs = []

matches = []

for offset in OFFSETS:

    # For each of set, it creates an url where we can find the matches links
    url = "https://www.hltv.org/results?offset="+str(offset)+"&content=demo&content=vod&team="+TEAM_ID
    URLs.append(url)

for URL in URLs:

    page = requests.get(URL,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    html = soup.find_all(class_='result-con')

    offset_matches = re.findall(r'href=\"/matches/(.*?)\"', str(html))
    matches.extend(offset_matches)

links = ["https://www.hltv.org/matches/"+sublink for sublink in matches]

df = pd.DataFrame(links, columns=['Link'])
df.drop_duplicates(inplace=True) # for some reason there's a value being duplicated during the offset parsing

download_links = []
dates = []
events = []
versus = []
picks = []

step = 0
stages = len(df['Link'].values)

print("{} matches found, starting to build HLTV\n".format(stages))

for link in df['Link'].values:  

    page = requests.get(link, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    try:

        # Scraping download links
        download_link = re.findall(r'href=\"/download/(.*?)\"', str(soup.find_all(class_='stream-box')))[0]
        download_links.append("https://www.hltv.org"+download_link+"/")

        # Scraping match dates
        date = re.findall(r'>(.*?)</div>', str(soup.find_all(class_='date')))[0]
        dates.append(date)

        # Scraping event name
        event = re.findall(r'title="(.*?)">', str(soup.find_all(class_='event text-ellipsis')))[0]
        events.append(event)

        # Scraping teams and setting up a "versus" format 
        team_1 = re.findall(r'title="(.*?)"/>', str(soup.find_all(class_='team1-gradient')))[0]
        team_2 = re.findall(r'title="(.*?)"/>', str(soup.find_all(class_='team2-gradient')))[0]

        versus.append(team_1 +" vs. "+team_2)

        # Scraping maps played
        maps = [x for x in re.findall(r'>(.*?)<', str(soup.find_all(class_='stats-menu-link')))[2:] if len(x) > 3]
        picks.append(', '.join(map(str, ax)))

    except:

        download_links.append(np.nan)
        dates.append(np.nan)
        events.append(np.nan)
        versus.append(np.nan)
        picks.append(np.nan)

    step += 1
    print("Table {:.1f}% completed...".format(step/stages * 100))

df['Download'] = download_links
df['Event'] = events
df['Date'] = pd.to_datetime(dates) 
df['Match'] = versus
df['Maps'] = picks

df.to_csv('data/downloads/furia_games_hltv.csv', index=False)

In [419]:


HLTV = 'data/downloads/furia_games_hltv.csv'
hltv = pd.read_csv(HLTV)
    
CURRENT_MATCHES = hltv['Link'].to_list()

matches = get_team_matches()['Link'].to_list()
outer_matches = [x for x in matches if x not in CURRENT_MATCHES]



In [422]:
matches[:3]

['https://www.hltv.org/matches/2346563/furia-vs-nip-iem-katowice-2021',
 'https://www.hltv.org/matches/2346225/mousesports-vs-furia-cs-summit-7',
 'https://www.hltv.org/matches/2346222/furia-vs-cloud9-cs-summit-7']

In [400]:
tt()

'Oi'

In [406]:
len(df['Link'].apply(lambda x: x.split('/')[-2]).unique())

359

In [ ]:
headers = {"User-Agent":
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36"}  

    # Team ID to refer

    TEAM_ID = "8297"

    # The team's match page is splitted through "offsets", the current threshold is 300 for average and to avoid possible old demos issues

    OFFSETS = [0, 100, 200, 300]

    URLs = []

    matches = []

    for offset in OFFSETS:

        # For each of set, it creates an url where we can find the matches links
        url = "https://www.hltv.org/results?offset="+str(offset)+"&content=demo&content=vod&team="+TEAM_ID
        URLs.append(url)
        
    for URL in URLs:

        page = requests.get(URL,headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')

        html = soup.find_all(class_='result-con')

        offset_matches = re.findall(r'href=\"/matches/(.*?)\"', str(html))
        matches.extend(offset_matches)